# Using EcoFOCIpy to process raw field data

## SKQ202312S (Arctic - Sikuliaq)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [1]:
import yaml
import glob

import EcoFOCIpy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import EcoFOCIpy.io.ncCFsave as ncCFsave
import EcoFOCIpy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2023/CTDcasts/skq202312s/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [3]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'SKQ202312S' #no hyphens
cruise_meta_file = sample_data_dir+'logs/SKQ202312S.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd002.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd009.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd012.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd014.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd016.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd018.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd019.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd021.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd023.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd025.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd026.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd027.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd028.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd029.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd030.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd031.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd032.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd033.btl


...

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd070.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd071.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd072.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd073.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd074.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd075.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd076.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd077.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude

Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd079.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd082.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/skq202312s/rawconverted/ctd084.btl


/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/sbe_ctd_parser.py:90: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  ctd_df = pd.concat([ctd_df,row])


In [4]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd001.btl'].describe()

,depsm,latitude,longitude,prdm,t090c,t190c,potemp090c,sal00,sal11,density00,sigma-t00,sbox0mm/kg,sbox1mm/kg,oxsolmm/kg,sbeox0ps,v0,fleco-afl,cstarat0,scan,datetime
count,23.00000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23
mean,33.06013,62.194535,-174.674006,33.384609,2.148291,2.149291,2.147461,31.352152,31.362665,1025.077452,24.916909,301.182043,288.716391,342.256249,88.687652,0.147100,0.534643,0.853443,24832.695652,2023-09-14 13:44:34.173913088
min,1.13500,62.194520,-174.674020,1.146000,-1.523100,-1.522400,-1.524100,30.776300,30.783700,1024.000700,23.995200,280.045000,267.238000,297.533560,75.406000,0.071000,0.078400,0.519400,9694.000000,2023-09-14 13:34:03
25%,10.12650,62.194530,-174.674015,10.224500,-1.518950,-1.517800,-1.519950,30.779050,30.787600,1024.042250,23.995900,283.417000,271.479000,297.583750,76.309000,0.074150,0.096800,0.709800,15980.500000,2023-09-14 13:38:25.500000
50%,30.13400,62.194540,-174.674000,30.429000,-1.292500,-1.293700,-1.293100,31.557500,31.572600,1025.518400,25.371600,287.982000,276.775000,369.908970,96.505000,0.144200,0.517500,0.744200,24445.000000,2023-09-14 13:44:18
75%,52.46000,62.194540,-174.674000,52.975500,7.746100,7.745550,7.745550,31.821250,31.831750,1025.846850,25.590950,303.819500,291.210500,371.383440,96.792500,0.211350,0.920200,1.107600,35104.000000,2023-09-14 13:51:42
max,66.93200,62.194540,-174.674000,67.592000,7.753400,7.754400,7.752500,31.837100,31.847700,1025.930600,25.603900,369.321000,354.880000,371.422870,99.844000,0.237200,1.075300,1.264200,40141.000000,2023-09-14 13:55:12
std,24.20635,0.000009,0.000009,24.445179,4.576586,4.576798,4.576635,0.483318,0.484956,0.866621,0.759069,29.658136,28.225841,36.536020,10.077116,0.066853,0.401098,0.267532,10937.100178,NaN


## Create BTL report file

In [5]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [6]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'SKQ202312S',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'R/V Sikuliaq',
 'ShipID': 'SKQ',
 'StartDate': '',
 'EndDate': '',
 'Project': '',
 'ChiefScientist': 'Seth Danielson',
 'StartPort': 'Dutch Harbor, AK',
 'EndPort': 'Nome, AK',
 'CruiseLocation': 'Arctic',
 'Description': '',
 'CruiseYear': 2023,
 'ctdlogs_pdf_name': 'SKQ202312S_CTDCastLogs.pdf'}

In [7]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 54636,
 'Vessel': 'R/V Sikuliaq',
 'CruiseID': 'SKQ202312S',
 'Project_Leg': '',
 'UniqueCruiseID': 'SKQ202312S',
 'Project': '',
 'StationNo_altname': '',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 62,
 'LatitudeMin': 11.69,
 'LongitudeDeg': 174,
 'LongitudeMin': 40.42,
 'GMTDay': 14,
 'GMTMonth': 'Sep',
 'GMTYear': 2023,
 'GMTTime': 47989,
 'DryBulb': -99.9,
 'RelativeHumidity': -999,
 'WetBulb': -99.9,
 'Pressure': '',
 'SeaState': '',
 'Visibility': '',
 'WindDir': -999,
 'WindSpd': -99.9,
 'CloudAmt': '',
 'CloudType': '',
 'Weather': '',
 'SurfaceTemp': -99.9,
 'BottomDepth': -9999,
 'StationNameID': '',
 'MaxDepth': -9999,
 'InstrumentSerialNos': '',
 'Notes': '',
 'NutrientBtlNiskinNo': '',
 'NutrientBtlNumbers': '',
 'OxygenBtlNiskinNo': '',
 'OxygenBtlNumbers': '',
 'SalinityBtlNiskinNo': '',
 'SalinityBtlNumbers': '',
 'ChlorophyllBtlNiskinNo': '',
 'ChlorophyllBtlVolumes': '',
 'InstrumentType': 'SBE911plus',
 'WaterMassCode': '',
 'isMooringProfileCast': 'n',
 

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [8]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [9]:
cruise_data = {k.upper():v for k,v in cruise_data.items()}

In [10]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['CTD001.BTL'] = cruise_data['CTD001.BTL'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['CTD001.BTL'].sample()

,depsm,latitude,longitude,Pressure [dbar],temperature_ch1,temperature_ch2,potemp090c,salinity_ch1,salinity_ch2,density00,...,oxy_conc_ch1,oxy_conc_ch2,oxsolmm/kg,oxy_percentsat_ch1,v0,chlor_fluorescence,Attenuation,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,,,,
14.0,25.161,62.19454,-174.674,25.406,-1.2504,-1.2414,-1.2508,31.4177,31.4291,1025.3799,...,369.321,354.88,369.89717,99.844,0.1982,0.8415,0.5775,28379.0,2023-09-14 13:47:02,ctd001


In [11]:
cruise_data['CTD001.BTL'].columns

Index(['depsm', 'latitude', 'longitude', 'Pressure [dbar]', 'temperature_ch1',
       'temperature_ch2', 'potemp090c', 'salinity_ch1', 'salinity_ch2',
       'density00', 'sigma_t_ch1', 'oxy_conc_ch1', 'oxy_conc_ch2',
       'oxsolmm/kg', 'oxy_percentsat_ch1', 'v0', 'chlor_fluorescence',
       'Attenuation', 'scan', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [12]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [13]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['CTD001.BTL'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [14]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 23)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 ... 20.0 21.0 22.0 23.0
Data variables: (12/21)
    depsm               (bottle) float64 66.93 66.77 66.93 ... 1.215 1.135 1.425
    latitude            (bottle) float64 62.19 62.19 62.19 ... 62.19 62.19 62.19
    longitude           (bottle) float64 -174.7 -174.7 -174.7 ... -174.7 -174.7
    Pressure [dbar]     (bottle) float64 67.59 67.42 67.59 ... 1.227 1.146 1.439
    temperature_ch1     (bottle) float64 -1.522 -1.522 -1.523 ... 7.75 7.747
    temperature_ch2     (bottle) float64 -1.522 -1.522 -1.522 ... 7.752 7.749
    ...                  ...
    v0                  (bottle) float64 0.0726 0.0734 0.0724 ... 0.2054 0.2106
    chlor_fluorescence  (bottle) float64 0.0878 0.092 0.0865 ... 0.8841 0.9158
    Attenuation         (bottle) float64 1.248 1.264 1.248 ... 0.7433 0.7442
    scan                (bottle) float64 9.694e+03 1.004e+04 ... 4.014e+04
    datetime            (bottle) datetime64[ns] 2023-09-14T13:34:03 ... 2023-...
    cast                (bottle) object 'ctd001' 'ctd001' ... 'ctd001' 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [20]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

ValueError: latitude already exists as coordinate or variable name.

In [16]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['CTD001.BTL'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [17]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 23, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 ... 20.0 21.0 22.0 23.0
  * time                (time) float64 1e+35
Data variables: (12/21)
    depsm               (time, bottle) float64 66.93 66.77 66.93 ... 1.135 1.425
    latitude            (time, bottle) float64 62.19 62.19 62.19 ... 62.19 62.19
    longitude           (time, bottle) float64 -174.7 -174.7 ... -174.7 -174.7
    Pressure [dbar]     (time, bottle) float64 67.59 67.42 67.59 ... 1.146 1.439
    temperature_ch1     (time, bottle) float64 -1.522 -1.522 ... 7.75 7.747
    temperature_ch2     (time, bottle) float64 -1.522 -1.522 ... 7.752 7.749
    ...                  ...
    v0                  (time, bottle) float64 0.0726 0.0734 ... 0.2054 0.2106
    chlor_fluorescence  (time, bottle) float64 0.0878 0.092 ... 0.8841 0.9158
    Attenuation         (time, bottle) float64 1.248 1.264 ... 0.7433 0.7442
    scan                (time, bottle) float64 9.694e+03 1.004e+04 ... 4.014e+04
    datetime            (time, bottle) datetime64[ns] 2023-09-14T13:34:03 ......
    cast                (time, bottle) object 'ctd001' 'ctd001' ... 'ctd001'

In [18]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 23, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 ... 20.0 21.0 22.0 23.0
  * time                (time) float64 1e+35
Data variables: (12/21)
    depsm               (time, bottle) float64 66.93 66.77 66.93 ... 1.135 1.425
    latitude            (time, bottle) float64 62.19 62.19 62.19 ... 62.19 62.19
    longitude           (time, bottle) float64 -174.7 -174.7 ... -174.7 -174.7
    Pressure [dbar]     (time, bottle) float64 67.59 67.42 67.59 ... 1.146 1.439
    temperature_ch1     (time, bottle) float64 -1.522 -1.522 ... 7.75 7.747
    temperature_ch2     (time, bottle) float64 -1.522 -1.522 ... 7.752 7.749
    ...                  ...
    v0                  (time, bottle) float64 0.0726 0.0734 ... 0.2054 0.2106
    chlor_fluorescence  (time, bottle) float64 0.0878 0.092 ... 0.8841 0.9158
    Attenuation         (time, bottle) float64 1.248 1.264 ... 0.7433 0.7442
    scan                (time, bottle) float64 9.694e+03 1.004e+04 ... 4.014e+04
    datetime            (time, bottle) datetime64[ns] 2023-09-14T13:34:03 ......
    cast                (time, bottle) object 'ctd001' 'ctd001' ... 'ctd001'
Attributes: (12/44)
    CruiseID:                      SKQ202312S
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        R/V Sikuliaq
    ShipID:                        SKQ
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2024-07-31T12:40:10Z
    date_modified:                 
    QC_indicator:                  unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [22]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'par/sat/log':'par',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})
        
        cruise_data[cast] = cruise_data[cast].drop(['latitude','longitude'],axis=1)
        
        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=True)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'seconds' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncCFsave.py:320: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'seconds' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(filename,format=kwargs['format'],encoding={'time':{'units':'days since 1900-01-01'}})
/Users/bell/src/ecofocipy/src/EcoFOCIpy/io/ncC

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history